In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import holidays
from sklearn.preprocessing import MinMaxScaler

2024-12-28 04:52:02.379284: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-28 04:52:02.875627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735361523.053274 1090521 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735361523.104188 1090521 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-28 04:52:03.534262: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import pandas as pd
import numpy as np
import holidays

def data_preprocessing(
    trips_parquet='filtered_df_2.parquet',
    clusters_csv='df_clusters.csv'
):
    """
    Loads the taxi trips and cluster info, merges them,
    computes pickup counts per cluster per day, 
    and adds cyclical day-of-week, month, holiday features.
    Returns df_daily.
    """
    # Load taxi trips
    df_trips = pd.read_parquet(trips_parquet)
    # Load cluster info
    df_clusters = pd.read_csv(clusters_csv)
    
    # Merge
    df_trips = df_trips.merge(df_clusters, left_on="PULocationID", right_on="zone_id", how="left")
    
    # Ensure datetime
    df_trips['tpep_pickup_datetime'] = pd.to_datetime(df_trips['tpep_pickup_datetime'])
    
    # Extract date
    df_trips['pickup_date'] = df_trips['tpep_pickup_datetime'].dt.date
    df_trips['pickup_date'] = pd.to_datetime(df_trips['pickup_date'])
    
    # Group by cluster & date
    df_daily = df_trips.groupby(['cluster', 'pickup_date']).size().reset_index(name='pickup_count')
    
    # Basic time features
    df_daily['day_of_week'] = df_daily['pickup_date'].dt.weekday  # Monday=0, Sunday=6
    df_daily['month']       = df_daily['pickup_date'].dt.month

    # Holiday indicator
    us_holidays = holidays.UnitedStates()
    df_daily['is_holiday'] = df_daily['pickup_date'].apply(lambda x: 1 if x in us_holidays else 0)
    
    # Cyclical encoding for day-of-week
    df_daily['dow_sin'] = np.sin(2 * np.pi * df_daily['day_of_week'] / 7)
    df_daily['dow_cos'] = np.cos(2 * np.pi * df_daily['day_of_week'] / 7)
    
    # Cyclical encoding for month
    df_daily['month_sin'] = np.sin(2 * np.pi * (df_daily['month'] - 1) / 12)
    df_daily['month_cos'] = np.cos(2 * np.pi * (df_daily['month'] - 1) / 12)
    
    # Sort by cluster & date
    df_daily = df_daily.sort_values(['cluster', 'pickup_date']).reset_index(drop=True)
    
    return df_daily

df_daily = data_preprocessing(
    trips_parquet='filtered_df_2.parquet',
    clusters_csv='df_clusters.csv'
)

df_daily

,cluster,pickup_date,pickup_count,day_of_week,month,is_holiday,dow_sin,dow_cos,month_sin,month_cos
0,0,2023-11-01,99771,2,11,0,0.974928,-0.222521,-0.866025,0.5
1,0,2023-11-02,102038,3,11,0,0.433884,-0.900969,-0.866025,0.5
2,0,2023-11-03,95066,4,11,0,-0.433884,-0.900969,-0.866025,0.5
3,0,2023-11-04,88657,5,11,0,-0.974928,-0.222521,-0.866025,0.5
4,0,2023-11-05,74430,6,11,0,-0.781831,0.623490,-0.866025,0.5
...,...,...,...,...,...,...,...,...,...,...
1595,4,2024-09-25,14,2,9,0,0.974928,-0.222521,-0.866025,-0.5
1596,4,2024-09-26,18,3,9,0,0.433884,-0.900969,-0.866025,-0.5
1597,4,2024-09-27,21,4,9,0,-0.433884,-0.900969,-0.866025,-0.5
1598,4,2024-09-28,23,5,9,0,-0.974928,-0.222521,-0.866025,-0.5
